# 下位机 JSON 指令集
在上一个章节中我们介绍了一个简单的例程，在例程中我们通过上位机向下位机发送运动控制指令，下位机可以接收的指令非常多，在这个章节中我们会介绍这些指令。

## JSON 指令的构成
以上一章节我们发送的 {"T":1,"L":0.2,"R":0.2} 指令为例，这个 JSON 数据中的 T 值代表指令的类型（Type），L 值代表左侧（LEFT）车轮的目标线速度，R 值代表右侧（RIGHT）车轮的目标线速度，线速度的单位默认为 m/s，总结起来的意思就是这是一条运动控制指令，运动控制的参数分别为左右两侧车轮的目标线速度。

后面的所有 JSON 指令，都会包含一个 T 值用于定义指令的类型，但是具体的指令参数会根据指令类型的不同而有所区别。

## JSON 指令集
你可以在我们开源的下位机例程的 json_cmd.h 文件中查看这些指令的定义，或自行为其添加新的下位机功能。
### 运动控制类指令
这些指令是移动机器人最基础的指令，用于运动相关的功能控制。

下面的指令中，每种指令包含三个部分：案列，简短的介绍和详细的介绍。

#### CMD_SPEED_CTRL
- {"T":1,"L":0.5,"R":0.5}
- 设置两侧车轮目标线速度（速度闭环控制）
> L 和 R 分别代表左右侧车轮的目标线速度，单位为 m/s，负值代变反向转动，0 为停止。目标线速度的取值范围取决于产品所使用的电机/减速器/车轮直径，相关的计算公式可以在开源的下位机例程中找到。这里需要注意的是，对于使用碳刷直流电机的底盘来说，当给定的目标速度的绝对值很小时（但不为 0），由于碳刷直流电机的低速性能通常比较差，所以产品在移动过程中速度可能波动比较大。
#### CMD_PWM_INPUT
- {"T":11,"L":164,"R":164}
- 设置两侧驱动轮的PWM值（速度开环控制）
> L 和 R 分别代表左右两侧车轮的 PWM 值，该值的范围为 -255 ~ 255，负值代表反向，当数值的绝对值为 255 时代表 PWM 为 100%，代表让这一侧的车轮满功率运行。
#### CMD_ROS_CTRL
- {"T":13,"X":0.1,"Z":0.3}
- ROS控制（速度闭环控制）
> 该指令用于 ROS 上位机控制底盘移动的指令，X 代表移动的线速度，单位为 m/s 可以为负值；Z 代表转向的角速度，单位为 rad/s 可以为负值。
#### CMD_SET_MOTOR_PID
- {"T":2,"P":20,"I":2000,"D":0,"L":255}
- PID控制器设置
> 该指令用于给PID控制器调参，上面 JSON 案例中的 PID 参数为该产品的默认参数，其中 L 代表 WINDUP_LIMITS，是预留的接口，目前的产品中暂时还没有使用到该参数。

### OLED 显示屏控制指令
产品上安装有 OLED 显示屏，该显示屏通过 I2C 与下位机主控 ESP32 模组进行通信，上位机可通过发送 JSON 指令来更改显示屏上显示的内容。
#### CMD_OLED_CTRL
- {"T":3,"lineNum":0,"Text":"putYourTextHere"}
- 控制显示屏显示自定义内容
> lineNum 是行数，一条 JSON 指令可以改变一行的内容，下位机收到一条新的指令后，开机默认的 OLED 界面会消失，取而代之的是你新增加的内容，对于大部分产品使用的 0.91英寸的OLED显示屏，lineNum 的数值可以为 0、1、2、3，共四行；Text 是你希望在这一行显示的文字内容，如果你这一行的内容太多，会自动换行，但同时也会挤掉最后一行。
#### CMD_OLED_DEFAULT
- {"T":-3}
- 控制显示屏显示自定义内容
> 使用这条指令让 OLED 显示屏显示开机默认的画面。

### 模块类型
移动底盘上面会安装不同类型的模块（无/机械臂/云台），使用这条指令来告诉下位机当前所安装的模块类型，这条指令通常会在上位机开机时自动发送给下位机，后面的章节会有这部分的介绍。
#### CMD_MODULE_TYPE
- {"T":4,"cmd":0}
- 设置模块类型
> cmd 的值代表模块的类型，目前有三种类型可以选择，0：什么都不装，1：机械臂，2：云台

### IMU 相关功能
底盘上安装有 IMU 传感器，你可以通过以下命令来获取 IMU 传感器的数据，这里需要注意的是，产品开机后会默认开启底盘信息连续反馈功能（其中包含有IMU的信息），这里的 IMU 相关功能仅在底盘信息连续反馈功能关闭时才有必要使用。
#### CMD_GET_IMU_DATA
- {"T":126}
- 获取 IMU 数据
> 发送该指令后可获取 IMU 的数据。
#### CMD_CALI_IMU_STEP
- {"T":127}
- IMU 校准(预留接口)
> 目前的产品程序不需要执行校准，此命令为预留接口。
#### CMD_GET_IMU_OFFSET
- {"T":128}
- 获取目前 IMU 的偏移量（预留接口）
> 使用该指令可反馈当前的 IMU 每个轴的偏移量。

#### CMD_SET_IMU_OFFSET

- {"T":129,"x":-12,"y":0,"z":0}
- 设置 IMU 的偏移量（预留接口）
> 使用该指令可以设置 IMU 每个轴的偏移量，此命令为预留接口，当前的产品不需要执行该指令。

### 底盘信息反馈

#### CMD_BASE_FEEDBACK
- {"T":130}
- 底盘信息反馈
> 产品开机后通常会默认开启底盘信息反馈，是自动的，如果底盘信息连续反馈功能关闭了，需要单次获取底盘的信息，可以使用该指令，可获取底盘的基础信息。

#### CMD_BASE_FEEDBACK_FLOW
- {"T":131,"cmd":1}
- 连续底盘信息反馈
> cmd的值设置为1，开启该功能，此功能默认开启，会持续反馈底盘信息；当cmd的值设置为0，关闭该功能，关闭该功能后，上位机可以通过 CMD_BASE_FEEDBACK 来获取底盘信息。

#### CMD_FEEDBACK_FLOW_INTERVAL
- {"T":142,"cmd":0}
- 设置连续反馈信息的格外间隔时间
> cmd的值为需要设置的格外间隔时间，单位为ms，通过该指令可以调整底盘反馈信息的频率。

#### CMD_UART_ECHO_MODE
- {"T":143,"cmd":0}
- 设置指令回声模式
> 当cmd的值设置为0时，关闭回声；当cmd的值设置为1时，开启回声，当开启指令回声模式后，下位机会输出接收到的指令。

### WIFI 相关设置

#### CMD_WIFI_ON_BOOT
- {"T":401,"cmd":3}
- 设置开机时的 WIFI 模式。
> cmd为0时，关闭WIFI功能；1-ap；2-sta；3-ap+sta。

#### CMD_SET_AP
- {"T":402,"ssid":"UGV","password":"12345678"}
- 设置AP模式的名称和密码。（ESP32建立热点）

#### CMD_SET_STA
- {"T":403,"ssid":"WIFI_NAME","password":"WIFI_PASSWORD"}
- 设置STA模式的名称和密码。（ESP32去连接已知热点）

#### CMD_WIFI_APSTA
- {"T":404,"ap_ssid":"UGV","ap_password":"12345678","sta_ssid":"WIFI_NAME","sta_password":"WIFI_PASSWORD"}
- 设置AP和STA模式的名称和密码。（AP+STA模式）

#### CMD_WIFI_INFO
- {"T":405}
- 获取当前WIFI的信息。

#### CMD_WIFI_CONFIG_CREATE_BY_STATUS
- {"T":406}
- 使用当前的设置新建一个WIFI配置文件。

#### CMD_WIFI_CONFIG_CREATE_BY_INPUT
- {"T":407,"mode":3,"ap_ssid":"UGV","ap_password":"12345678","sta_ssid":"WIFI_NAME","sta_password":"WIFI_PASSWORD"}
- 使用输入的设置新建一个 WIFI 配置文件。

#### CMD_WIFI_STOP
- {"T":408}
- 断开 WIFI 连接。

### 12V开关和云台设置

#### CMD_LED_CTRL
- {"T":132,"IO4":255,"IO5":255}
- 12V开关输出设置
> 产品下位机上面有两路12V开关接口，每路2个接口共四个接口，你可以通过这个指令来设置这些接口的输出电压，当数值为255时为3S电池电压。
> 产品默认使用这些接口来控制LED灯，你可以通过这个指令来控制LED灯的亮度。

#### CMD_GIMBAL_CTRL_SIMPLE
- {"T":133,"X":0,"Y":0,"SPD":0,"ACC":0}
- 云台基础控制指令
> 该指令用于控制云台的朝向。
> X为水平方向的朝向，单位为角度，正值向右，负值向左，取值范围为 -180到180。
> Y为数值方向的朝向，单位为角度，正值向上，负值向下，取值范围为 -30到90。
> SPD为速度，ACC为加速度，当值为0时为最快速度/加速度。

#### CMD_GIMBAL_CTRL_MOVE
- {"T":134,"X":45,"Y":45,"SX":300,"SY":300}
- 云台连续控制指令
> 该指令用于连续控制云台朝向。
> X为水平方向的朝向，单位为角度，正值向右，负值向左，取值范围为 -180到180。
> Y为数值方向的朝向，单位为角度，正值向上，负值向下，取值范围为 -30到90。
> SX和SY分别为X轴和Y轴的速度。

#### CMD_GIMBAL_CTRL_STOP
- {"T":135}
- 云台停止指令
> 当使用上面的命令让云台运动起来后，可以使用该指令让云台可以随时停下来。

#### CMD_GIMBAL_STEADY
- {"T":137,"s":0,"y":0}
- 云台自稳功能
> s为0时关闭该功能，s为1时开启该功能，该功能开启后云台会自动通过IMU数据来调整云台的竖直角度，y为云台的与地面的目标夹角（即使开启云台自稳功能，摄像头也可以上下看）。

#### CMD_GIMBAL_USER_CTRL
- {"T":141,"X":0,"Y":0,"SPD":300}
- 云台UI控制
> 该指令用于UI界面控制云台，X值可以为-1，0和1，-1为向左转动，0为停止，1为向右转动。
> Y值可以为-1，0和1，-1为向下转动，0为停止，1为向上转动。
> SPD为速度

### 机械臂控制

#### CMD_MOVE_INIT
- {"T":100}
- 机械臂运动到初始姿态
> 正常情况下机械臂开机时会自动转动到初始位置。 该指令会引起进程阻塞。

#### CMD_SINGLE_JOINT_CTRL
- {"T":101,"joint":0,"rad":0,"spd":0,"acc":10}
- 机械臂单轴运动控制
> joint：关节序号。
1：BASE_JOINT基础关节。
2：SHOULDER_JOINT肩关节。
3：ELBOW_JOINT肘关节。
4：EOAT_JOINT手腕/夹爪关节。
rad：为需要转动到的角度（以弧度制形式来显示），以各个关节的初始位置为准，各个关节的默认角度和转动方向如下：
BASE_JOINT的初始位置默认角度为 0，角度转动范围为 3.14 至 -3.14 之间。角度增加时，基础关节向左转动；角度减少时，基础关节向右转动。
SHOULDER_JOINT的初始位置默认角度为 0，角度转动范围为 1.57 至 -1.57 之间。角度增加时，肩关节向前转动；角度减少时，肩关节会向后转动。
ELBOW_JOINT的初始位置默认角度为 1.570796，角度转动范围为 3.14 至 -1.11 之间。角度增加时，肘关节向下转动；角度减少时，肘关节会往反方向转动。
EOAT_JOINT的初始位置默认角度为 3.141593。产品默认的是夹爪关节，角度转动范围为 1.08 至 3.14 之间，角度减少时，夹爪关节会张开。若更换为手腕关节，角度转动范围为 1.08 至 5.20 之间，角度增加时，手腕关节向下转动；角度减少时，手腕关节向上转动。
spd：转动的速度，速度单位为步/秒，舵机的一圈为 4096 步，数值越大速度越快，当速度值为 0 时，以最大速度转动。
acc：转动开始和结束时的加速度，数值越小启停越平滑，数值可以为 0-254 ，单位为 100 步/秒^2。如设置为 10 时，则按照 1000 步每秒的平方加速度变速。当加速度值为 0 时，则

#### CMD_JOINTS_RAD_CTRL
- {"T":102,"base":0,"shoulder":0,"elbow":1.57,"hand":1.57,"spd":0,"acc":10}
- 以弧度制形式来控制机械臂全部关节的转动
> base：基础关节的角度，角度转动范围见上方“CMD_SINGLE_JOINT_CTRL”指令中rad键的说明。
shoulder：肩关节的角度。
elbow：肘关节的角度。
hand：夹爪/手腕关节的角度。
spd：转动的速度，速度单位为步/秒，舵机的一圈为4096步，数值越大速度越快，当速度值为0时，以最大速度转动。
acc：转动开始和结束时的加速度，数值越小启停越平滑，数值可以为0-254，单位为100步/秒^2。如设置为10时，则按照1000步每秒的平方加减速度变速。当加速度值为0时，则按照最大的加

#### CMD_SINGLE_AXIS_CTRL
- {"T":103,"axis":2,"pos":0,"spd":0.25}
- 单轴坐标控制
> axis,1-x,2-y,3-z,4-t。除T轴外，其它轴的pos参数为mm，T轴的单位为rad。spd是速度系数，数值越高速度越快。

#### CMD_XYZT_GOAL_CTRL
- {"T":104,"x":235,"y":0,"z":234,"t":3.14,"spd":0.25}
- 机械臂坐标运动控制（逆运动学控制）
> 该函数会引发阻塞

#### CMD_XYZT_DIRECT_CTRL
- {"T":1041,"x":235,"y":0,"z":234,"t":3.14}
- 机械臂坐标运动控制（逆运动学控制）
> 该函数不会引发阻塞

#### CMD_SERVO_RAD_FEEDBACK
- {"T":105}
- 反馈机械臂的坐标信息

#### CMD_EOAT_HAND_CTRL
- {"T":106,"cmd":1.57,"spd":0,"acc":0}
- 末端关节控制（弧度制）
> cmd：为需要转动到的角度（以弧度制来显示）。EOAT_JOINT的初始位置默认角度为3.141593。
产品默认的是夹爪关节，角度转动范围为1.08至3.14之间，角度减少时，夹爪关节会张开。
若更换为手腕关节，角度转动范围为1.08至5.20之间，角度增加时，手腕关节向下转动；角度减少时，手腕关节向上转动。
spd：转动的速度，速度单位为步/秒，舵机的一圈为4096步，数值越大速度越快，当速度值为0时，以最大速度转动。
acc：转动开始和结束时的加速度，数值越小启停越平滑，数值可以为0-254，单位为100步/秒^2。如设置为10时，则按照1000步每秒的平方加减速度变速。当加速度值为0时，则按照最大的加速

#### CMD_EOAT_GRAB_TORQUE
- {"T":107,"tor":200}
- 夹爪力量控制
> tor的值最高可以为1000，代表100%的力量。

#### CMD_SET_JOINT_PID
- {"T":108,"joint":3,"p":16,"i":0}
- 关节PID设置

#### CMD_RESET_PID
- {"T":109}
- 重置关节PID

#### CMD_SET_NEW_X
- {"T":110,"xAxisAngle":0}
- 设置新的X轴方向

#### CMD_DYNAMIC_ADAPTATION
- {"T":112,"mode":0,"b":1000,"s":1000,"e":1000,"h":1000}
- 动态外力自适应控制

### 其它设置

#### CMD_HEART_BEAT_SET
- {"T":136,"cmd":3000}
- 设置心跳函数时间
> cmd单位为ms，可以使用该指令设置心跳函数时间，如果下位机在该时间内没有接收到新的运动指令，会自动停止运动，用于避免使用过程中上位机死机导致下位机一直运动下去。
程中上位机死机导致下位机一直运动下去。

#### CMD_SET_SPD_RATE
- {"T":138,"L":1,"R":1}
- 设置左右速度比率
> 产品使用差速转向原理，当产品左右车轮给相同的目标速度时，产品有可能由于编码器误差或轮胎抓地力误差导致产品不走直线，你可以使用该指令来对左右侧车轮的速度进行微调，例如左边的车轮需要转动慢一些的话，可以将L的数值改为 0.98。L值和R值尽量不要设置大于一的数值。

#### CMD_GET_SPD_RATE
- {"T":139}
- 获取当前的速度比率
> 使用该指令可以获取当前的速度比例



### ESP-NOW 相关设置

#### CMD_BROADCAST_FOLLOWER
- {"T":300,"mode":1}
- {"T":300,"mode":0,"mac":"CC:DB:A7:5B:E4:1C"}
- 设置ESP-NOW被广播控制的模式
> mode为1时，可以让其它设备通过广播指令控制；mode为0时只能由mac地址的设备控制。

#### CMD_GET_MAC_ADDRESS
- {"T":302}
- 获取当前设备的MAC地址

#### CMD_ESP_NOW_ADD_FOLLOWER
- {"T":303,"mac":"FF:FF:FF:FF:FF:FF"}
- 将MAC地址添加到被控设备（PEER）

#### CMD_ESP_NOW_REMOVE_FOLLOWER
- {"T":304,"mac":"FF:FF:FF:FF:FF:FF"}
- 将MAC地址从PEER中删除

#### CMD_ESP_NOW_GROUP_CTRL
- {"T":305,"dev":0,"b":0,"s":0,"e":1.57,"h":1.57,"cmd":0,"megs":"hello!"}
- ESP-NOW 组播控制

#### CMD_ESP_NOW_SINGLE
- {"T":306,"mac":"FF:FF:FF:FF:FF:FF","dev":0,"b":0,"s":0,"e":1.57,"h":1.57,"cmd":0,"megs":"hello!"}
- ESP-NOW 单播/组播控制

### 任务文件相关功能

该功能属于下位机的高阶功能，正常带上位机使用时通常不需要进行以下的操作。

#### CMD_SCAN_FILES
- {"T":200}
- 扫描当前的任务文件

#### CMD_CREATE_FILE
- {"T":201,"name":"file.txt","content":"inputContentHere."}
- 新建任务文件

#### CMD_READ_FILE
- {"T":202,"name":"file.txt"}
- 读取任务文件

#### CMD_DELETE_FILE
- {"T":203,"name":"file.txt"}
- 删除任务文件

#### CMD_APPEND_LINE
- {"T":204,"name":"file.txt","content":"inputContentHere."}
- 在任务文件末尾添加新的指令

#### CMD_INSERT_LINE
- {"T":205,"name":"file.txt","lineNum":3,"content":"content"}
- 在任务文件中间插入新的指令

#### CMD_REPLACE_LINE
- {"T":206,"name":"file.txt","lineNum":3,"content":"Content"}
- 替换任务文件中的某一个指令

### 总线舵机设置

#### CMD_SET_SERVO_ID
- {"T":501,"raw":1,"new":11}
- 更改舵机ID
> raw是舵机的原始ID（新舵机都是1），new是要更改为的ID，最大不超过254，不可为负，255为广播ID。

#### CMD_SET_MIDDLE
- {"T":502,"id":11}
- 将该舵机的目前位置设置为舵机中位（仅ST系列舵机有效）。

#### CMD_SET_SERVO_PID
- {"T":503,"id":14,"p":16}
- 设置舵机的PID的P值。

### ESP32 相关功能

#### CMD_REBOOT
- {"T":600}
- 重启。

#### CMD_FREE_FLASH_SPACE
- {"T":601}
- 获取 FLASH 的剩余空间大小。

#### CMD_BOOT_MISSION_INFO
- {"T":602}
- 输出当前的开机任务文件。

#### CMD_RESET_BOOT_MISSION
- {"T":603}
- 重置开机任务文件。

#### CMD_NVS_CLEAR
- {"T":604}
- 清理 ESP32 的 NVS 区，如果不能成功WIFI连接，可以尝试调用该指令，然后重启。

#### CMD_INFO_PRINT
- {"T":605,"cmd":1}
- 设置信息反馈模式。
> cmd为1时，打印输出调试信息；2，连续的底盘信息反馈；0，什么都不反馈。

#### CMD_MM_TYPE_SET
- {"T":900,"main":2,"module":0}
- 设置底盘类型和底盘模块类型
> main:  
> 1 - RaspRover  
> 2 - UGV Rover  
> 3 - UGV Beast  
> module:  
> 0 - None  
> 1 - RoArm  
> 2 - Pan-Tilt  